# Regularization

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import RobustScaler

import sklearn
%matplotlib inline

import ipywidgets as widgets
from tqdm.notebook import tqdm

import warnings
# silence future deprecation warnings
warnings.filterwarnings('ignore')

## Part 1: Regularization for Linear Regresion

### Prepare the data

Although linear regression is a linear machine learning method, you can have nonlinear dependencies if you transform some of the independent variables by a nonlinear function. By doing this, you can improve the fit of your method. Let us demonstrate this on a house price dataset from [Kaggle](https://www.kaggle.com/harlfoxem/housesalesprediction). Note that this dataset is not identical with one you used in the linear regression exercise, since the this dataset is too small and would cause unreliable evaluation results.

In [ ]:
df = pd.read_csv("kc_house_data.csv")
df.head()

We would like to have a simple linear regression problem with only one independent variable. Thus, we only keep *price* and *sqft_living*.

In [ ]:
df = df[["price","sqft_living"]]
df.head()

#### Split the data

We split the data into a training and test set

In [ ]:
train, test = train_test_split(df, test_size=0.5, random_state=42)

#### Normalize the data
Let us normalize the data by using *min-max normalization*

In [ ]:
scaler = MinMaxScaler()

train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)
test = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)

train.head()

In [ ]:
X_train = train[["sqft_living"]]
y_train = train[["price"]]

X_test = test[["sqft_living"]]
y_test = test[["price"]]

### Bias term
To account for the bias term, we add a column containing only ones.

In [ ]:
X_train["bias"] = 1
X_test["bias"] = 1

# Force order
X_train = X_train[["bias", "sqft_living"]]
X_test = X_test[["bias", "sqft_living"]]

X_train.head()

### Fit a linear regression model
Define a linear regression function to estimate the parameters $\theta$ based on the normal equation:
  
  $\Theta:=(X^{\top}X)^{-1}(X^{\top}y)$

In [ ]:
def fit(X, y):
    # START YOUR CODE
    
    # END YOUR CODE
    return thetas

In [ ]:
def fit(X, y):
    thetas = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
    return thetas

Run the following code to check your implementation:

In [ ]:
thetas = fit(X_train, y_train)

expected_thetas = np.array([[7.39560812e-05], [4.94185750e-01]])
np.testing.assert_array_almost_equal(thetas, expected_thetas, decimal=4)

### Predict prices
Using $X$ and the estimated $\theta$, predict the house prices on the training data

In [ ]:
def predict(X, thetas):
    # START YOUR CODE
    
    # END YOUR CODE
    return y_pred

In [ ]:
def predict(X, thetas):
    y_pred = np.dot(X, thetas)
    return y_pred

In [ ]:
y_pred = predict(X_train, thetas)
y_pred

### Visualize predictions
Let us plot house prices and predicted house prices

In [ ]:
def plot_regression_line(X, thetas, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    deg = len(thetas)-1
    poly = PolynomialFeatures(deg)
    
    xs = np.arange(X.min(), X.max()+0.1, 0.01).reshape(-1,1)
    x = poly.fit_transform(xs)
    y_pred = np.dot(x, thetas)
    
    ax.plot(xs, y_pred, color="r")

In [ ]:
fig, ax = plt.subplots()
ax.plot(X_train["sqft_living"].values, y_train.values, "bo", markersize=1)
plot_regression_line(X_train["sqft_living"].values, thetas, ax)

### Calculate model performance
Now let's check how good our model performs by calculating the $R^2$ score on the test set.

In [ ]:
# r2 = ...

In [ ]:
y_pred_test = predict(X_test, thetas)
r2 = r2_score(y_test, y_pred_test)
print("R2: ",r2)

### Adding polynomial features

We aim to improve the fit by adding $x^2$ as additional independent variable.

In [ ]:
X_train_deg2 = X_train.copy()
X_train_deg2["sqft_living^2"] = X_train_deg2["sqft_living"] * X_train_deg2["sqft_living"]

X_test_deg2 = X_test.copy()
X_test_deg2["sqft_living^2"] = X_test_deg2["sqft_living"] * X_test_deg2["sqft_living"]
X_test_deg2.head()

#### Fit the model with the additonal features

In [ ]:
thetas_deg2 = fit(X_train_deg2, y_train)

#### Calculate the performance

In [ ]:
# r2 = 

In [ ]:
y_pred_test = predict(X_test_deg2, thetas_deg2)
r2 = r2_score(y_test, y_pred_test)
print("R2: ",r2)

As we can see, by adding $x^2$ as additional independent variable we could slightly improve our performance.

Let's try if we can further improve our performance by adding more polynomial features. To generate our polynomial features we will use the Scikit-Learn function [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html). 

In [ ]:
@widgets.interact(poly_deg =(1,18,1))
def f(poly_deg=1):
    poly = PolynomialFeatures(poly_deg)
    X_train_deg = poly.fit_transform(X_train["sqft_living"].values.reshape(-1,1))
    X_test_deg = poly.fit_transform(X_test["sqft_living"].values.reshape(-1,1))

    thetas_deg = fit(X_train_deg, y_train)
    
    y_pred_test = predict(X_test_deg, thetas_deg)
    y_pred_train = predict(X_train_deg, thetas_deg)
    
    r2_test = r2_score(y_test, y_pred_test)
    r2_train = r2_score(y_train, y_pred_train)
    print("R2 Train {0:.5f}".format(r2_train))
    print("R2 Test {0:.5f}".format(r2_test))
    
    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(20,10))
    ax0.set_title("Training data - polynomial degree {}".format(poly_deg))
    ax0.plot(X_train["sqft_living"], y_train["price"], "bo", markersize=1)
    plot_regression_line(X_train_deg, thetas_deg, ax0)
    
    ax1.set_title("Test data - polynomial degree {}".format(poly_deg))
    ax1.plot(X_test["sqft_living"], y_test["price"], "bo", markersize=1)
    plot_regression_line(X_test_deg, thetas_deg, ax1)

What do you recognize when you increase the polynomial degree?

> Answer the question on ILIAS

### Regularization

The effect of overfitting can be reduced by regularization. Implement the regularized version of linear regression: $\Theta:=(X^{\top}X+\lambda \begin{bmatrix}
    0  & 0 &\ldots&0 \\
    0 & 1 & \\
    \ldots & & \ddots & \\
    0& & & 1
  \end{bmatrix} )^{-1}(X^{\top}y)$

In [ ]:
def fit_reg(X, y, lam):
    # START YOUR CODE

    # END YOUR CODE
    return thetas

In [ ]:
def fit_reg(X, y, lam):
    Xt = np.transpose(X)
    XtX = np.dot(Xt,X)
    XtX = XtX + (lam * np.identity(XtX.shape[0]))
    XtXm1 = np.linalg.inv(XtX)
    Xty = np.dot(Xt,y)
    thetas = np.dot(XtXm1,Xty)
    return thetas

You can check your implementation by executing the following cell:

In [ ]:
expected_thetas = np.array([[0.0017878 ], [0.48483375]])
actual_thetas = fit_reg(X_train, y_train, lam=2)

np.testing.assert_array_almost_equal(expected_thetas, actual_thetas)

We  plot the graph using the regularized parameter vectors. As you can see, the effect of overfitting is strongly reduced.

In [ ]:
@widgets.interact(poly_deg = (0,12,1), lam=(0,100,1))
def f(poly_deg=1, lam=4):
    poly = PolynomialFeatures(poly_deg)
    X_train_deg = poly.fit_transform(X_train["sqft_living"].values.reshape(-1,1))
    X_test_deg = poly.fit_transform(X_test["sqft_living"].values.reshape(-1,1))

    thetas_deg = fit_reg(X_train_deg, y_train, lam=lam)
    
    y_pred_test = predict(X_test_deg, thetas_deg)
    y_pred_train = predict(X_train_deg, thetas_deg)
    
    r2_test = r2_score(y_test, y_pred_test)
    r2_train = r2_score(y_train, y_pred_train)
    print("R2 Train", r2_train)
    print("R2 Test", r2_test)
    
    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(20,10))
    ax0.set_title("Training data - polynomial degree {}".format(poly_deg))
    ax0.plot(X_train["sqft_living"], y_train["price"], "bo", markersize=1)
    plot_regression_line(X_train_deg, thetas_deg, ax0)
    
    ax0.set_title("Test data - polynomial degree {}".format(poly_deg))
    ax1.plot(X_test["sqft_living"], y_test["price"], "bo", markersize=1)
    plot_regression_line(X_test_deg, thetas_deg, ax1)

Find the best configuration of **polynomial degree** and $\lambda$

<font color='red'>PLEASE REPLACE TEXT WITH YOUR CONFIGURATION</font>

### Regularization to help with numerical issues

Another benefit of regularization is that it can help in case of numerical issues. Let us consider our original dataset.

In [ ]:
df = pd.read_csv("kc_house_data.csv")
df = df[["price","sqft_living","bedrooms"]]
df.head()

In [ ]:
train, test = train_test_split(df, test_size=0.5, random_state=42)

In [ ]:
scaler = MinMaxScaler()

train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)
test = pd.DataFrame(scaler.fit_transform(test), columns=test.columns, index=test.index)

test.head()

To make the feature matrix $X^{\top}X$ singular, we just add  another independent variable (Size2) to X
that amounts to just twice the Size.

In [ ]:
train["sqft_living2"] = 2 * train["sqft_living"]
train["bias"] = 1

test["sqft_living2"]=2 * test["sqft_living"]
test["bias"] = 1

test.head()

In [ ]:
X_train = train[["bias", "sqft_living", "bedrooms", "sqft_living2"]]
y_train = train[["price"]]

X_test = test[["bias", "sqft_living", "bedrooms", "sqft_living2"]]
y_test = test[["price"]]

We see that the linear regression fails, since $X^{\top}X$ is not invertible.

In [ ]:
thetas = fit(X_train, y_train)

There are two possiblities to tackle this issue, the first one is to use the pseudoinverse instead of the inverse
and the second one is using regularization. 

> Try out both. 

*Hint*: For conducting linear regression with the pseudoinverse, you have to slightly modify the linear_regression method given further above. 
The numpy function [np.linalg.pinv](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.inv.html) becomes handy for this.

In [ ]:
def fit_pseudoinverse(X,y):
    # START YOUR CODE
    
    # END YOUR CODE
    return thetas

In [ ]:
def fit_pseudoinverse(X, y):
    thetas = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)
    return thetas

Run this code to check your implementation

In [ ]:
thetas_pseudo_inverse = fit_pseudoinverse(X_train, y_train)
print ("thetas obtained by linear regression with pseudoinverse:\n",thetas_pseudo_inverse)

expected_thetas_pseudo_inverse = np.array([
    [ 0.02902459],
    [ 0.11220321],
    [-0.12253607],
    [ 0.22440641]])

np.testing.assert_array_almost_equal(thetas_pseudo_inverse, expected_thetas_pseudo_inverse, decimal=5)

In [ ]:
thetas_regularization = fit_reg(X_train, y_train,lam=1)
print ("thetas obtained by linear regression with regularization:\n",thetas_regularization)

expected_thetas_regularization = np.array([
    [ 0.02842384],
    [ 0.11163472],
    [-0.11920766],
    [ 0.22326945]])

np.testing.assert_array_almost_equal(thetas_regularization, expected_thetas_regularization, decimal=5)

## Part 2: Regularization for Logistic Regression 

Now let us apply regularization to a larger dataset using sklearn, for which we reuse the eczema dataset from the last exercise.

In [ ]:
df = pd.read_csv("skin_disease.csv")
df.drop('t0', axis=1, inplace=True)
df.drop('t1', axis=1, inplace=True)

We again split the data into training and test set.

In [ ]:
X = df.drop(columns=["class"])
y = df[["class"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

We scale the data using the RobustScaler.

In [ ]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Fit a Logistic Regression model
> Train a Logistic Regression model by using the Scikit-Learn class [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). And calculate the accuracy and the f1 score on the test data.

In [ ]:
# START YOU CODE


# END YOUR CODE

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred_test = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)

print("Accuracy", accuracy)
print("F1 Score", f1)

### Regularize our Logistic Regression model
> For the logistic regression model we can also apply L2 regularization. Try systematically all L2 regularization parameters in the interval [0,2] with step size 0.1, a technique also known as grid search. Calculate for each iteration the accuracy and the f1 score.

Hint: Take a look at the documentation of the [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) class and specify the correct **regularization type** with the parameter **penalty**.

[GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [ ]:
accuracies = []
f1_scores = []
C_values = np.arange(0.1, 2.1, 0.1)
for c in tqdm(C_values):
    # START YOUR CODE
    #model = LogisticRegression(...)
    
    
    pass
    # END YOUR CODE
    
max_acc_idx = np.argmax(accuracies)
print("Best accuracy: {0:.4f} with C-value: {1:.2f}".format(accuracies[max_acc_idx], C_values[max_acc_idx]))
print("Best f1_score: {0:.4f} with C-value: {1:.2f}".format(f1_scores[max_acc_idx], C_values[max_acc_idx]))

In [ ]:
accuracies = []
f1_scores = []
C_values = np.arange(0.1, 2.1, 0.1)
for c in tqdm(C_values):
    model = LogisticRegression(penalty="l2", C=c)
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_test)
    f1 = f1_score(y_test, y_pred_test)
    
    accuracies.append(accuracy)
    f1_scores.append(f1)
    
max_acc_idx = np.argmax(accuracies)
print("Best accuracy: {0:.4f} with C-value: {1:.2f}".format(accuracies[max_acc_idx], C_values[max_acc_idx]))
print("Best f1_score: {0:.4f} with C-value: {1:.2f}".format(f1_scores[max_acc_idx], C_values[max_acc_idx]))

### Grid-Search with Scikit-Learn
This can be also done directly with Scikit-Learn. The following cell performs a grid search using a 5-fold cross validation.

<font color='red'>This can take a while!</font>

In [ ]:
model = LogisticRegression(penalty="l2")
params = {
    'C': np.arange(0.1, 2.1, 0.1)
}
grid_search = GridSearchCV(model, params, cv=5, scoring="f1")
grid_search.fit(X_train, y_train)
print("Best params", grid_search.best_params_)
print("Best f1 score", grid_search.best_score_)

We can see that the best C value is not the same as we got when we performed our grid search manually. The reason is that we did not use cross validation but tuned our hyperparameters on the test set. This should not be done because then we overfit our model on the test set!

One final remark, in this notebook we used for simplicity skin pixels from the same image for training and testing.
In real world scenarios, the test and training data set should originate from different images, 
since skin pixels from the same image are highly correlated and your evaluation measure values would therefore 
probably be too high.

## Assignment
> Now finish the ILIAS quiz to **Regularization**.